<a href="https://www.kaggle.com/code/maorsagi/clustering-2019?scriptVersionId=109725935" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 2015 Elections Data

# Imports

In [ ]:
!pip install --upgrade git+https://github.com/goolig/dsClass.git
from dsClass.path_helper import *
import dsClass

In [ ]:
import pandas as pd
import itertools
from datetime import timedelta
import os
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

import seaborn as sns
sns.set_context("poster")
plt.style.use('fivethirtyeight')
#plt.style.use('ggplot')
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['figure.titleweight'] = 'bold' 

from IPython.display import display, HTML

import numpy as np
import math

import datetime
import time
import sys

import networkx as nx


import sklearn
print("sklearn.__version__:",sklearn.__version__)

import pylab as pl
import matplotlib.dates as mdates

#print('Machine: {} {}\n'.format(os.uname().sysname,os.uname().machine))
print(sys.version)
from dsClass.path_helper import *

# Data Import

In [ ]:
file_name = get_file_path("results_by_booth_2015 - english - v3_utf8.csv")
result_df = pd.read_csv(file_name,delimiter=',')
print("Columns:")
print(result_df.columns)
print()
print("df shape:",result_df.shape)
result_df

# Clean Data

In [ ]:
result_df = result_df.dropna(axis=0, how='any')
result_df = result_df[result_df.votes > 0]
result_df.loc[result_df.Registered_voters == 0,'Registered_voters'] = result_df[result_df.Registered_voters == 0].votes
result_df.shape

# Overall Votes Per Party

In [ ]:
block_percent = 0.0325

In [ ]:
all_registered_voters = result_df.Registered_voters.sum()
all_votes = result_df.proper_votes.sum()
print("all registerd voters:",all_registered_voters)
print("all_votes:",all_votes)
print("vote percentage:",all_votes/all_registered_voters)
overall_votes_per_party = result_df.iloc[:,8:].sum()
percantage_vote_per_pary = overall_votes_per_party/all_votes
percantage_vote_per_pary = percantage_vote_per_pary[percantage_vote_per_pary.values>block_percent]
percantage_vote_per_pary.sort_values(ascending=False).plot.bar(alpha=0.7)

# Group by City and Filter Out Small Parties

In [ ]:
# Print the large parties
large_parties = percantage_vote_per_pary.index.values
print(large_parties)

In [ ]:
non_party_col = list(result_df.iloc[:,0:8].columns)
int_columns = []
int_columns.extend(non_party_col)
int_columns.extend(list(large_parties))
print(int_columns)

In [ ]:
res_work_df = result_df.copy()
res_work_df = res_work_df[int_columns]
res_work_df_city = res_work_df.groupby(['settlement_name_english','Settlement_code'])[int_columns[4:]].sum().reset_index()
print(res_work_df_city.shape)
res_work_df_city.head(5)

# Remove low votings rates

In [ ]:
min_vote_rate = 0.6
min_proper_votes = 300

In [ ]:
res_work_df = res_work_df_city.copy()
res_work_df['vote_rate'] = res_work_df.proper_votes / res_work_df.Registered_voters
res_work_df = res_work_df[(res_work_df.vote_rate > min_vote_rate) & (res_work_df.proper_votes > min_proper_votes)]
print(res_work_df.shape)
res_work_df.sample(10)

In [ ]:
res_work_df[res_work_df.settlement_name_english.str.contains("BE'ER SHEVA|TEL AVIV|JERU|HAI")] # BE'ER SHEVA / TEL AVIV / JERUSALEM 


## Check if there bad rows with infinite values

In [ ]:
res_work_df[res_work_df.vote_rate == np.inf]

# Calculate percentage votes for each city-party

In [ ]:
res_work_df_percentage_votes = res_work_df.iloc[:,6:-1].div(res_work_df.proper_votes, axis=0)
res_work_df_percentage_votes.head(5)

# Clustering

In [ ]:
res_work_df_percentage_votes_transpose = res_work_df_percentage_votes.transpose()
res_work_df_percentage_votes_transpose.head(11)

## Run K-Means
- Tanspose matrix
- Convert numeric voting rate to (1,0) where 1 means the voting rate in that settelment was above the median

In [ ]:
X = res_work_df_percentage_votes_transpose
X.head(3)

In [ ]:
def above_median(fclist):
    med = np.median(fclist)
    return (fclist > med).astype(int)

# מטריקה - האם גבוה מהמדיאן או לא

X = X.apply(above_median, axis=1)

In [ ]:
X[1:10]

In [ ]:
names = res_work_df_percentage_votes_transpose.index.tolist()

In [ ]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=4, random_state=0).fit(X)
clusters = km.labels_.tolist()
clusters

## Visualize Clusters

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2)
results_tsne = tsne.fit(X)

coords = results_tsne.embedding_

colors = ['blue','red','green','cyan','magenta','yellow','black','white']
label_colors = [colors[i] for i in clusters]

plt.subplots_adjust(bottom = 0.1)
plt.scatter(
    coords[:, 0], coords[:, 1], marker = 'o', c=label_colors
    )

#names = distance.index.values.tolist()
for label, x, y in zip(names, coords[:, 0], coords[:, 1]):
    plt.annotate(
        label,
        xy = (x, y), xytext = (-20, 20),
        textcoords = 'offset points', ha = 'right', va = 'bottom',
        bbox = dict(boxstyle = 'round,pad=0.5', fc = 'yellow', alpha = 0.5),
        arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0'))

plt.show()


# Distance Matrix
- http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html
- https://docs.scipy.org/doc/scipy/reference/spatial.distance.html#module-scipy.spatial.distance

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import MinMaxScaler

x = res_work_df_percentage_votes_transpose

res = pairwise_distances(x, metric='correlation') # cosine / jaccard / correlation / euclidean

distance = pd.DataFrame(res, index=res_work_df_percentage_votes_transpose.index, 
                        columns= res_work_df_percentage_votes_transpose.index)

distance

## Hierarchical Clustering

In [ ]:
import scipy
from scipy.cluster import hierarchy

labels = distance.index.values.tolist()
sq_distance = scipy.spatial.distance.squareform(distance)

Z = hierarchy.linkage(sq_distance, 'single')
#plt.figure()
#dn = hierarchy.dendrogram(Z)

hierarchy.set_link_color_palette(['m', 'c', 'y', 'k'])
fig, axes = plt.subplots(1, 1, figsize=(16, 6))
#dn1 = hierarchy.dendrogram(Z, ax=axes[0], above_threshold_color='y', orientation='top', labels=labels)
dn1 = hierarchy.dendrogram(Z, ax=axes, above_threshold_color='y', orientation='top', labels=labels)
#dn2 = hierarchy.dendrogram(Z, ax=axes[1], above_threshold_color='#bcbddc', orientation='right', labels=labels)
hierarchy.set_link_color_palette(None)  # reset to default after use
plt.show()

## Heatmap of Distance Matrix Reordered as the Dendrogram

In [ ]:
new_order_distance = distance.reindex(dn1['ivl'])
new_order_distance = new_order_distance[dn1['ivl']] 

In [ ]:
import seaborn as sns
ax = sns.heatmap(new_order_distance)

# Build Network

In [ ]:
distance_cutoff = 1
parties = percantage_vote_per_pary.index.tolist()
parties

In [ ]:
import itertools
dist_list = list(distance.index)
all_2_org_combos = itertools.combinations(dist_list, 2)
max_dist = distance.max().max()

# Generate graph with nodes: 
G=nx.Graph()

for p in parties:
    G.add_node(p,
               name=p,
               p_vote=float(percantage_vote_per_pary[p]),             
               comm="0") 
    
# Connect nodes:
for combo in all_2_org_combos:
    combo_dist = distance[combo[0]][combo[1]]
    opp_dist = combo_dist - max_dist
    if distance[combo[0]][combo[1]] < distance_cutoff:
        G.add_edge(combo[0],combo[1],weight=float(abs(opp_dist)))
        

n = G.number_of_nodes()
m = G.number_of_edges()     
print("number of nodes in graph G: ",n)
print("number of edges in graph G: ",m)
print()

## Communities and Modularity
- https://en.wikipedia.org/wiki/Modularity_%28networks%29
- https://en.wikipedia.org/wiki/Community_structure
- https://en.wikipedia.org/wiki/Louvain_Modularity

In [ ]:
import community
communities = community.best_partition(G)
mod = community.modularity(communities,G)
print("modularity:", mod)

In [ ]:
if m > 0:         
    for k,v in communities.items():
        #for n in C:
        G.node[k]['comm'] = str(v)
else:
    print("Not runnig Community algorithm because the graph has no edges")

## Draw Networka
- https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.drawing.nx_pylab.draw_networkx.html
- https://networkx.github.io/documentation/networkx-1.10/reference/drawing.html

In [ ]:
com_values = [communities.get(node) for node in G.nodes()]

p_votes = [d['p_vote'] for n,d in G.nodes(data=True)]
node_size=[v * 3000 for v in p_votes]

#pos=nx.spring_layout(G)
#pos=nx.spectral_layout(G)
#pos=nx.random_layout(G)
#pos=nx.circular_layout(G)
#pos = nx.shell_layout(G)
pos=nx.fruchterman_reingold_layout(G)
nx.draw_networkx(G,pos, cmap = plt.get_cmap('jet'), node_color = com_values, node_size=node_size, with_labels=True)

## Generate gexf file to view in Gephi
- Save network to a gexf file format
- Upload file in Gephi
- Order layout using: OpenOrd / Yifan Hu / Fruchterman Reingold
- Appearance of nodes:
  - Color nodes by community
  - Node size by number of votes (proper votes)
  - Label size by number of votes as well

In [ ]:
graphFileName = "parties.gexf"
nx.write_gexf(G, graphFileName, encoding='utf-8')

# Now Generate a Similar Network For Cities 

## Distance Matrix
- The distance will be between each city - try the correlation metric first
- The vector of each city will be composed of the relative votes for each party

In [ ]:
res_work_df_percentage_votes.head()

## Build Distance matrix and view it

In [ ]:


#We would like to measure distance by correlation
matrix_calc = pairwise_distances(res_work_df_percentage_votes, metric='correlation') 

distance = pd.DataFrame(matrix_calc, index=res_work_df_percentage_votes.index, 
                        columns= res_work_df_percentage_votes.index)
distance.head()


## Build Network

### Build the network of cities with the following attributes
- distance_cutoff = 0.01
- min_proper_votes = 1000 : use only cities where proper_votes > min_proper_votes
- Data for each node: city code, city name, proper votes, community
- Edge weight: similarity 

In [ ]:
distance_cutoff = 0.01
min_proper_votes = 1000

### Create dictionaries if neccassary

In [ ]:
cities_list = list(res_work_df.index)
dict_idx_names = {}
dict_names_properVotes = {}
length = len(res_work_df.index)

for i in range(length):
    dict_idx_names[cities_list[i]] = res_work_df.loc[cities_list[i]]["settlement_name_english"]
for i in range(length):
    dict_names_properVotes[dict_idx_names[cities_list[i]]] = res_work_df.loc[cities_list[i]]["proper_votes"]
    
print("dict_idx_names[9] :",dict_idx_names[9])
print("dict_names_properVotes[ABU SINAN] :",dict_names_properVotes['ABU SINAN'])

### Create list of cities that hold all the constraints
- How many cities hold the constrains?

In [ ]:
cities_idx_1000 = []
for i in range(length):
    if dict_names_properVotes[dict_idx_names[cities_list[i]]] > min_proper_votes:
        cities_idx_1000.append(cities_list[i])

print(len(cities_idx_1000))  

### Network Generation
- How many nodes?
- How many edges?

In [ ]:
all_2_org_combos = itertools.combinations(cities_idx_1000, 2)
max_dist = distance.max().max()

# Generate graph with nodes: 
G=nx.Graph()

for c in cities_idx_1000:
    G.add_node(c,
               name=dict_idx_names[c],
               p_vote=float(dict_names_properVotes[dict_idx_names[c]]),             
               comm="0") 
# Connect nodes:
for combo in all_2_org_combos:
    combo_dist = distance.loc[combo[0]][combo[1]]
    opp_dist = combo_dist - max_dist
    if distance[combo[0]][combo[1]] < distance_cutoff:
        G.add_edge(combo[0],combo[1],weight=float(abs(opp_dist)))
        

n = G.number_of_nodes()
m = G.number_of_edges()     
print("number of nodes in graph G: ",n)
print("number of edges in graph G: ",m)


### Run community algorithm
- How many communities?
- How many nodes in the largest community?
- What is the modularity score?

In [ ]:
communities = community.best_partition(G)
mod = community.modularity(communities,G)


track={}

for key,value in communities.items():
    if value not in track:
        track[value]=0
    else:
        track[value]+=1

print("number of communities:", len(track))
print("largest community:", max(track,key=track.get))
print("modularity:", mod)

### Add communitites information to the nodes in the network

In [ ]:
if m > 0:         
    for k,v in communities.items():
        #for n in C:
        G.node[k]['comm'] = str(v)
else:
    print("Not runnig Community algorithm because the graph has no edges")

## Draw Network
- Set node size according to votes / degree
- Try different layouts

In [ ]:
# node_size=[v * 0.01 for v in p_votes]


com_values = [communities.get(node) for node in G.nodes()]
p_votes = [d['p_vote'] for n,d in G.nodes(data=True)]
node_size=[v*0.01 for v in p_votes]
G = nx.relabel_nodes(G,dict_idx_names)
plt.figure(90,figsize=(40,40))
#pos=nx.spring_layout(G)
#pos=nx.spectral_layout(G)
#pos=nx.random_layout(G)
#pos=nx.circular_layout(G)
#pos = nx.shell_layout(G)
pos=nx.fruchterman_reingold_layout(G)
nx.draw_networkx(G,pos, cmap = plt.get_cmap('jet'), node_color = com_values, node_size=node_size, with_labels=True,font_size=8)

## Calculate:

### Sum over network
- Using the network, sum proper votes for each community 
- Print the top 5 communities (in sum of votes) and the cities that compose them
- For one of the cities in a top-5 community, print the votes for each party 

In [ ]:
dict_com_pr_votes = {}
dict_com_cities = {}
comm_list =[d for n,d in list(G.nodes(data='comm'))]
p_vote_list = [d for n,d in list(G.nodes(data='p_vote'))]
cities = [d for n,d in list(G.nodes(data='name'))]
if m > 0:         
    for i in  range(len(comm_list)):
        comm =comm_list[i]
        vote = p_vote_list[i]
        city = cities[i]
        if comm in dict_com_pr_votes.keys():
            dict_com_pr_votes[comm] += vote
        else:
            dict_com_pr_votes[comm] = vote
        if comm in dict_com_cities.keys():
            dict_com_cities[comm].append(city)
        else:
            dict_com_cities[comm]=[city]
else:
    print("Not runnig algorithm because the graph has no edges")
    
import operator
#max_values = max(dict_com_pr_votes,5)   
sorted_dict = sorted(dict_com_pr_votes.items(), key=operator.itemgetter(1), reverse=True)[:5]
for i in range(5):
    comm = sorted_dict[i][0]
    print("Community Name:",comm," Cities",dict_com_cities[comm],"\n")

res_work_df[res_work_df.settlement_name_english.str.contains("JERU")][['settlement_name_english','Joint List', 'Kulanu', 'Likud', 'Meretz', 'Shas', 'The Jewish Home', 'United Torah Judaism', 'Yesh Atid', 'Yisrael Beiteinu', 'Zionist Union']]

